In [11]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import transformers

# cfg

In [4]:
class CFG:
    model_test=True
    frac=0.005#0.02 #1.0#
    pretrained=True #False#
    model= "tf_efficientnet_b0"#"resnet18"#"tf_efficientnet_b0"#"resnet18"
    train_bs=4
    valid_bs=4#12
    test_bs=4
    epoch=5
    fold=[0]
    seed= 2022
    lr=2e-4
    es=4
    device="cuda"

    # make fold
    make_fold=0
    make_clean=1
    
    #train, inference
    _train=0
    _predict=0

    #file path
    # path_train="E:/kaggle_data/Jigsaw Rate Severity of Toxic Comments/jigsaw-toxic-comment-classification-challen/train.csv"
    # path_train_test="E:/kaggle_data/Jigsaw Rate Severity of Toxic Comments/jigsaw-toxic-comment-classification-challen/test.csv"
    # path_train_test_label="E:/kaggle_data/Jigsaw Rate Severity of Toxic Comments/jigsaw-toxic-comment-classification-challen/test_labels.csv"
    # path_test="E:/kaggle_data/Jigsaw Rate Severity of Toxic Comments/validation_data.csv"
    # path_test2="E:/kaggle_data/Jigsaw Rate Severity of Toxic Comments/comments_to_score.csv"
    
    path_train="../input/jigsaw-toxic-comment-classification-challenge/train.csv"
    path_train_test="../input/jigsaw-toxic-comment-classification-challeng/test.csv"
    path_train_test_label="../input/jigsaw-toxic-comment-classification-challeng/test_labels.csv"
    path_test="../input/jigsaw-toxic-severity-rating/sample_submission.csv"
    path_test2="../input/jigsaw-toxic-severity-rating/comments_to_score.csv"

    #customize
    max_sens=128

def get_cfg():
    return CFG

# preprocess

In [5]:
from sklearn.model_selection import StratifiedKFold

def clean(data):

    # Clean some punctutations
    data = re.sub('\n', ' ', data)
    # Remove ip address
    data = re.sub(r'(([0-9]+\.){2,}[0-9]+)',' ', data)
    
    data = re.sub(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', data)
    # Replace repeating characters more than 3 times to length of 3
    data = re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1', data)
    # patterns with repeating characters 
    data = re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1', data)
    data = re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', data)

    # Add space around repeating characters
    data = re.sub(' +', ' ', data)
    
    # Ex) I didn ' t -> I didn't
    data = re.sub(" ' ", "'", data)
    
    return data

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text    

def get_data():
    test=pd.read_csv(CFG.path_train_test)
    test_label=pd.read_csv(CFG.path_train_test_label).replace(-1,0)
    test_merge=pd.merge(test,test_label,how="left",on="id")
    train=pd.read_csv(CFG.path_train)
    train=pd.concat([train,test_merge])

    cats = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}
    for cat in cats:
        train[cat] = train[cat] * cats[cat]
    train["target"]=(train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1))

    min_len=train[train.target>0].shape[0]
    under_sample=train[train.target==0].sample(n=min_len,random_state=CFG.seed)
    train=pd.concat([train[train.target>0],under_sample]).reset_index()

    train["cat"]=(train.target*100).astype(int)
    if CFG.make_fold:
        skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=5)
        train["fold"]=-1
        for i,(_,val_idx) in enumerate(skf.split(train,train.cat)):
            train.loc[val_idx,"fold"]=i 
    
    if CFG.make_clean:
        train.comment_text=train.comment_text.apply(lambda x: text_cleaning(x))

    return train    

def get_data_test():
    comments_to_score = pd.read_csv(CFG.path_test2)
    comments_to_score.columns=["comment_id","comment_text"]
    comments_to_score["target"]=-1
    if CFG.make_clean:
        comments_to_score.comment_text=comments_to_score.comment_text.apply(lambda x: text_cleaning(x))
    return comments_to_score

# main

In [12]:
    #train data
    train = get_data()

    # TF-IDF
    vec=TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (2,5),max_features=46000)
    X=vec.fit_transform(train.comment_text)

    # Model
    model=Ridge(alpha=0.5)
    model.fit(X,train["target"])

    #submission
    test = get_data_test()
    print(test.head())
    X_test=vec.transform(test.comment_text)
    test["score"]=model.predict(X_test)
    test[["comment_id","score"]].to_csv("submission.csv",index=False)
    print(test.head())

                      id                                       comment_text  \
159561  ffd2e85b07b3c7e4  "\nNo he did not, read it again (I would have ...   
159562  ffd72e9766c09c97  "\n Auto guides and the motoring press are not...   
159563  ffe029a7c79dc7fe  "\nplease identify what part of BLP applies be...   
159564  ffe897e7f7182c90  Catalan independentism is the social movement ...   
159565  ffe8b9316245be30  The numbers in parentheses are the additional ...   
159566  ffe987279560d7ff  ":::::And for the second time of asking, when ...   
159567  ffea4adeee384e90  You should be ashamed of yourself \n\nThat is ...   
159568  ffee36eab5c267c9  Spitzer \n\nUmm, theres no actual article for ...   
159569  fff125370e4aaaf3  And it looks like it was actually you who put ...   
159570  fff46fc426af1f9a  "\nAnd ... I really don't think you understand...   

        toxic  severe_toxic  obscene  threat  insult  identity_hate  target  \
159561      0             0        0       0       

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyboardInterrupt: 